# Quick Start for Network Wrangler

This notebook should give you a high-level overview of using Wrangler to do the following:  

1. Import a network to wrangler
2. Selections - which are used to determine which part of a network to apply a change to
3. Changes to attributes 
4. Write out networks  
5. Project cards to define selections, changes, and metadata
6. Scenarios to manage groups of projects

In [1]:
import warnings
from pathlib import Path

from network_wrangler import load_roadway_from_dir, write_roadway, load_transit, write_transit
from projectcard import read_card

%config IPCompleter.greedy=True
import pandas as pd
pd.set_option("display.max_columns", None)
warnings.filterwarnings("ignore")

import logging

logger = logging.getLogger("WranglerLogger")
# set to logging.INFO or DEBUG depending on how much detail you want in the logs
logger.setLevel(logging.INFO)

# 1. Importing and viewing a network
## Locate network folder and files 

In [2]:
STPAUL_DIR = Path.cwd().parent / "examples" / "stpaul"

## Roadway Network

The roadway network is comprised of three tables
1. Nodes: `nodes_df` with `model_node_id` primary key
2. Links: `links_df` with `model_link_id` primary key
   - Columns `A` and `B` are foreign keys to `nodes_df.model_node_id` representing from-node and to-node respectfully.
   - Column  `shape_id` is a foreign key to `shapes_df.shape_id`
2. Shapes: `shapes_df` with  `shape_id` primary key

### Network Read
 - Read in the network specifying where each network component can be found
 

In [26]:
road_net = load_roadway_from_dir(STPAUL_DIR)

Skipping field outboundReferenceIds: unsupported OGR type: 5
Skipping field inboundReferenceIds: unsupported OGR type: 5


### Explore the roadway network

* look at first three rows of each network component

In [4]:
road_rnet.links_df[:3]

,model_link_id,osm_link_id,shstReferenceId,shstGeometryId,shape_id,u,v,A,B,locationReferences,distance,roadway,name,ref,bridge,tunnel,width,max_speed,bike_facility,drive_access,walk_access,bike_access,truck_access,bus_only,rail_only,lanes,access,price,trn_priority,ttime_assert,geometry,projects,managed,ML_projects
model_link_id_idx,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
224,224,"18214905,18220565",6a22969708104ae2497244f3d079381d,4a454269d65c6619a2d7e2026295a295,9ef30f714e972087771ed705654fd1f5,493882040,187879468,3230,52771,"[{'sequence': 1, 'point': [-93.0837695, 44.963...",0.050322,residential,"Arkwright Street,North Rivoli Street",,,,,,3,False,True,True,False,False,False,1,,0.0,0,0,"LINESTRING (-93.08377 44.96334, -93.08324 44.9...",,0,
280,280,18218485,bff76735795c148707c774d7f7a0fa6b,13e68dc0fe4477f10b9655ebbd5f8435,2ac3bd68a0d0a2d5e6c4f7dcfc19a36e,493882338,493882339,3261,3262,"[{'sequence': 1, 'point': [-93.0855338, 44.966...",0.070096,tertiary,East Cayuga Street,,,,,,0,True,False,False,True,False,False,2,,0.0,0,0,"LINESTRING (-93.08553 44.96621, -93.08431 44.9...",,0,
281,281,"221685888,221685889,221685893",42b68a489b91dd5a1415fb25fb53de65,9ba14c0a7330177aa938ad0ec43b45f1,f6b3bee60aac5786dc35c6d51f7dab43,493882338,2307229054,3261,131209,"[{'sequence': 1, 'point': [-93.0855338, 44.966...",0.084513,tertiary,East Cayuga Street,,yes,,,,0,True,False,False,True,False,False,2,,0.0,0,0,"LINESTRING (-93.08553 44.96621, -93.08712 44.9...",,0,


In [5]:
road_net.nodes_df[:3]

,model_node_id,osm_node_id,shstReferenceId,drive_node,walk_node,bike_node,bus_only,rail_only,geometry,X,Y,projects
model_node_id_idx,,,,,,,,,,,,
1924,1924,1924,,1,1,1,0,0,POINT (-93.13665 44.96983),-93.136654,44.969831,
1925,1925,1925,,1,1,1,0,0,POINT (-93.13713 44.96468),-93.137132,44.964678,
1930,1930,1930,,1,1,1,0,0,POINT (-93.14154 44.95373),-93.141544,44.953727,


In [6]:
road_net.shapes_df[:3]

,id,shape_id,fromIntersectionId,toIntersectionId,forwardReferenceId,geometry
shape_id_idx,,,,,,
9ef30f714e972087771ed705654fd1f5,4a454269d65c6619a2d7e2026295a295,9ef30f714e972087771ed705654fd1f5,4d0231aa0ebb779f142c2518703ee481,3654951b676940911fe5021b93c90cc5,6a22969708104ae2497244f3d079381d,"LINESTRING (-93.08377 44.96334, -93.08376 44.9..."
2ac3bd68a0d0a2d5e6c4f7dcfc19a36e,13e68dc0fe4477f10b9655ebbd5f8435,2ac3bd68a0d0a2d5e6c4f7dcfc19a36e,b6ee5e5a4adca2f379b20fe7ee7ca77d,51013ccb2e52ee1ea83de6d36e4ef268,bff76735795c148707c774d7f7a0fa6b,"LINESTRING (-93.08553 44.96621, -93.08524 44.9..."
f6b3bee60aac5786dc35c6d51f7dab43,9ba14c0a7330177aa938ad0ec43b45f1,f6b3bee60aac5786dc35c6d51f7dab43,b6ee5e5a4adca2f379b20fe7ee7ca77d,863278524c056439c207888e8f09d04e,42b68a489b91dd5a1415fb25fb53de65,"LINESTRING (-93.08553 44.96621, -93.08574 44.9..."


## Transit Network

Transit network `feed` property has components represent a dataframe for each GTFS file including:

 - `routes` keyed on `route_id`
 - `stops` keyed on `stop_id`
 - `trips` keyed on `trip_id`
 - `stop_times` with foreign keys `trip_id` and `stop_id`
 - `frequencies` with foreign key `trip_id`

### Read Network

Specify the path where to find the transit network files

In [7]:
transit_net = load_transit(STPAUL_DIR)

### Explore the transit network

* look at the first few components of each transit network component

In [8]:
transit_net.feed.routes[:3]

,route_id,agency_id,route_short_name,route_long_name,route_desc,route_type,route_url,route_color,route_text_color
0,3-111,0,3,U of M - Como Av - Energy Park Dr - Maryland Av,None,3,http://www.metrotransit.org/route/3,None,000000
1,16-111,2,16,University Av - Midway,None,3,http://www.metrotransit.org/route/16,None,000000
2,21-111,0,21,Uptown - Lake St - Midway - Selby Av,None,3,http://www.metrotransit.org/route/21,None,000000


In [9]:
transit_net.feed.stops[:3]

,stop_id,stop_code,stop_name,stop_desc,stop_lat,stop_lon,zone_id,stop_url,location_type,wheelchair_boarding,osm_node_id,node_ssid,parent_station,stop_id_GTFS,projects,gtfs_stop_id
0,3261,None,Westminster St & Cayuga St W,Near side S,44.966288,-93.085602,None,http://www.metrotransit.org/NexTripBadge.aspx?...,0,1,493882338,b6ee5e5a4adca2f379b20fe7ee7ca77d,None,None,,53785
1,4219,None,12th St W & John Ireland Blvd,Near side W,44.951319,-93.104096,None,http://www.metrotransit.org/NexTripBadge.aspx?...,0,1,954746986,9a057e241974a5839b6bf9c805892197,None,None,,56164
2,5149,None,Wabasha St & Vision Loss Resources,Near side N,44.936163,-93.086662,None,http://www.metrotransit.org/NexTripBadge.aspx?...,0,1,652245100,55d02fa3aa91908ecba6798814bd9d73,None,None,,"291,47040"


In [10]:
transit_net.feed.trips[0:3]

,route_id,service_id,trip_id,trip_headsign,direction_id,block_id,shape_id,wheelchair_accessible,trip_destination,trip_short_name,projects,trip_route_name,pub_dir_id,rm_block_id
0,452-111,JUN19-MVS-BUS-Weekday-01,14940701-JUN19-MVS-BUS-Weekday-01,Westbound 452 Express / Minneapolis,1,1874,4520004,1,None,None,,NaN,None,None
1,3-111,JUN19-MVS-BUS-Weekday-01,14941148-JUN19-MVS-BUS-Weekday-01,Eastbound 3B Como Av/St Paul - Depot/Via Front,0,1018,30009,1,None,None,,NaN,None,None
2,3-111,JUN19-MVS-BUS-Weekday-01,14941151-JUN19-MVS-BUS-Weekday-01,Westbound 3B Como-Front/Dwtn Mpls/Target Field,1,1017,30014,1,None,None,,NaN,None,None


In [11]:
transit_net.feed.stop_times[:3]

,trip_id,arrival_time,departure_time,stop_sequence,pickup_type,drop_off_type,stop_distance,timepoint,stop_is_skipped,stop_id,projects
0,14941148-JUN19-MVS-BUS-Weekday-01,NaT,NaT,46,0,0,NaN,0,NaN,72333,
1,14941148-JUN19-MVS-BUS-Weekday-01,2024-10-16 00:01:00,2024-10-16 00:01:00,47,0,0,NaN,0,NaN,83136,
2,14941148-JUN19-MVS-BUS-Weekday-01,2024-10-16 00:01:00,2024-10-16 00:01:00,48,0,0,NaN,0,NaN,66544,


In [12]:
transit_net.feed.frequencies[0:3]

,trip_id,headway_secs,start_time,end_time,projects
0,14940701-JUN19-MVS-BUS-Weekday-01,3600,2024-10-16 06:00:00,2024-10-16 09:00:00,
1,14941148-JUN19-MVS-BUS-Weekday-01,830,2024-10-16 06:00:00,2024-10-16 09:00:00,
2,14941151-JUN19-MVS-BUS-Weekday-01,540,2024-10-16 06:00:00,2024-10-16 09:00:00,


In [13]:
transit_net.feed.shapes[:5]

,shape_id,shape_pt_lat,shape_pt_lon,shape_pt_sequence,shape_osm_node_id,shape_model_node_id,projects
0,4520004,44.923257,-93.067060,55,507955865,152495,
1,4520004,44.930632,-93.071166,56,507951624,151635,
2,4520004,44.932669,-93.071710,57,507951633,151637,
3,4520004,44.939445,-93.075273,58,507951645,151641,
4,4520004,44.943056,-93.076998,59,2548150957,23834,


### Map Transit Network

There are several convenience methods to transit network to create geodataframes to make the transit network straightforward to visualize.

- `transit_net.stops_gdf` Stops from `stops.txt`
- `shapes_gdf` Consolidate shapes from `shapes.txt`
- `shape_links_gdf` Individual links from `shapes.txt`
- `stop_time_links_gdf` Links from `stop_times.txt`
- `stop_times_points_gdf` Points from `stop_times.txt`

In [14]:
my_map = transit_net.stops_gdf.explore(
    tooltips=["stop_id","stop_name"],
    color = "purple",
    tiles = "cartodbpositron"
)
transit_net.stop_time_links_gdf.explore(
    tooltips=["trip_id"],
    m = my_map,
    color = "grey",
)


**View a routes or a single route**

In [16]:
selected_route_ids = ["3-111"]
from network_wrangler.transit.feed.stop_times import stop_times_for_route_ids
from network_wrangler.transit.geo import stop_times_to_stop_time_links_gdf
sel_stop_times = stop_times_for_route_ids(transit_net.feed.stop_times, transit_net.feed.trips, selected_route_ids)
sel_stop_times_gdf = stop_times_to_stop_time_links_gdf(sel_stop_times, transit_net.feed.stops)
sel_stop_times_gdf.explore(m=my_map, color="red", tooltips=["trip_id", "stop_id"])

## 2. Query Network Features

Most of Network wrangler operates by querying a portion of the network and then making changes to it.  

Queries take the form of Python Dictionaries or can be read-in as YAML which is then converted to a python dictionary.

If a query has already been made, it is stored in the network so that it will not take up computing power to find it again.

### Highway Segment Selection

Highway selections for links have three required components: `links`, `from` and `to`.  

- `links` must either have a specified name, or an 'osmid'
- `from` and `to` must specify some sort of unique identifier which is found in the data structure (AKA, it should return a single node) like `model_node_id` or `osm_node_id`.

If not all the links connecting `from` to `to` have the `name`, wrangler will connect them using as many streets with the correct name as possible.

In [31]:
easy_selection = {
    "links": {
            "name": ["6th", "Sixth", "sixth"]
    },  # find streets that have one of the various forms of 6th
    "from": {"osm_node_id": "187899923"},  # start searching for segments at this id
    "to": {"osm_node_id": "187942339"},  # end segment at this id
}

In [32]:
# querying with the selection will return the links that are selected
sel = road_net.get_selection(easy_selection)
display_cols = ["name", "A", "B", "lanes"]
sel.selected_links_df[display_cols]

,name,A,B,lanes
model_link_id_idx,,,,
2846,East 6th Street,4758,4806,1
2918,East 6th Street,4806,72303,1
78740,East 5th Street,39430,68608,3
96983,East 5th Street,46665,150855,3
134542,East 5th Street,62145,39430,2
148977,East 5th Street,68608,76167,1
157437,East 6th Street,72303,77072,1
165718,East 5th Street,76167,46665,3
167745,East 6th Street,77072,77073,1


In [34]:
sel.selected_links_df.explore(tiles="cartodbpositron", column = "lanes", cmap = "viridis", tooltip=display_cols)

### More complex selections

You can also select facilities based on their properties.  This selection is tiered from the name/osmid selection.

In [42]:
multi_criteria_selection = {
    "links": {
        "name": ["6th", "Sixth", "sixth"],
        "lanes": [2, 3],  # from the initial selection, only select streets with 1 OR 2 lanes
    },
    "from": {"osm_node_id": "187899923"},  # start searching for segments at this id
    "to": {"osm_node_id": "187942339"},  # end segment at this id
}
multi_sel = road_net.get_selection(multi_criteria_selection)
multi_sel.selected_links_df.explore(tiles="cartodbpositron", column = "lanes", cmap = "viridis",tooltip=display_cols)

### Transit Selection


Transit trip selection can currently happen by querying trips, routes, etc.

In [37]:
selection_by_trip_prop = {"trip_properties": {"route_id": ["365-111"]}}
selected_trips = transit_net.get_selection(selection_by_trip_prop).selected_trips
transit_net.feed.trips[transit_net.feed.trips.trip_id.isin(selected_trips)]

,route_id,service_id,trip_id,trip_headsign,direction_id,block_id,shape_id,wheelchair_accessible,trip_destination,trip_short_name,projects,trip_route_name,pub_dir_id,rm_block_id
35,365-111,JUN19-MVS-BUS-Weekday-01,14947182-JUN19-MVS-BUS-Weekday-01,Northbound 365 Express / Minneapolis,0,1844,3650001,1,None,None,,NaN,None,None


In [39]:
selection_by_route_prop  = {"route_properties": {"route_short_name": ["3"]}}
selected_trips = transit_net.get_selection(selection_by_route_prop).selected_trips
transit_net.feed.routes[transit_net.feed.trips.trip_id.isin(selected_trips)]

,route_id,agency_id,route_short_name,route_long_name,route_desc,route_type,route_url,route_color,route_text_color
1,16-111,2,16,University Av - Midway,None,3,http://www.metrotransit.org/route/16,None,000000
2,21-111,0,21,Uptown - Lake St - Midway - Selby Av,None,3,http://www.metrotransit.org/route/21,None,000000
3,53-111,0,53,Ltd Stop - Uptown - Lake St - Marshall Av,None,3,http://www.metrotransit.org/route/53,None,000000
4,62-111,0,62,Shoreview - Little Canada - Rice St Signal Hills,None,3,http://www.metrotransit.org/route/62,None,000000


#### Visualize selection on map

Note that this is a mouthful of code.  I have a todo to write a wrapper around this so that you can easily visualize with an api of something like:  `transit_net.get_selection(simple_transit_selection).plot()`

In [45]:
from network_wrangler.transit.feed.stop_times import stop_times_for_trip_ids
from network_wrangler.transit.geo import stop_times_to_stop_time_links_gdf
sel_stop_times = stop_times_for_trip_ids(transit_net.feed.stop_times, selected_trips)
sel_stop_times_gdf = stop_times_to_stop_time_links_gdf(sel_stop_times, transit_net.feed.stops)
sel_stop_times_gdf.explore(tiles="cartodbpositron", color="green", tooltips=["trip_id", "stop_id"])

# 3. Change Feature Attributes

Changes are made by:
1. selecting the features to change
2. defining what properties should change and how

Selections are defined by dictionaries containing selection requirements. 

## Highway Feature Change

 - RoadwayNetwork link features are selected by link features, and conditionally start and end points
 - Additional refinements to the selection can made by specifying additional properties such as `lanes` etc.
 

In [76]:
selected_highway = {
    "links": {"ref": ["I 35E"]},
}

### Specify Change

Changes are specified by a dictionary.  For highway changes, there should be a list of dictionaries under `properties` where each entity has a `property` and either `set` for an absolute value or `change`  to direct that a value is changed. 

In [77]:
lane_change = {
    "project": "lane_change",
    "roadway_property_change": {
        "facility": selected_highway, 
        "property_changes": {
            "lanes": {"set": 2}
        }
    }
}


In [81]:
road_net_build = road_net.apply(lane_change)
road_net_build.links_df.loc[road_net_build.links_df.ref == "I 35E", ["ref","lanes","projects"]].head()

,ref,lanes,projects
model_link_id_idx,,,
381412,I 35E,2,"lane_change,lane_change,lane_change,lane_change,"
382547,I 35E,2,"lane_change,lane_change,lane_change,lane_change,"
385471,I 35E,2,"lane_change,lane_change,lane_change,lane_change,"
390634,I 35E,2,"lane_change,lane_change,lane_change,lane_change,"
390975,I 35E,2,"lane_change,lane_change,lane_change,lane_change,"


#### You can also increment from the existing value using the field `change`
And optionally, you can call out what you think the existing value should be so that it fails if it isn't the case.

In [82]:
lane_change = {
    "project": "lane_change",
    "roadway_property_change": {
        "facility": selected_highway, 
        "property_changes": {
            "lanes": {"change": 4}
        }
    }
}

road_net_build = road_net.apply(lane_change)
road_net_build.links_df.loc[road_net_build.links_df.ref == "I 35E", ["ref","lanes","projects"]].head()

,ref,lanes,projects
model_link_id_idx,,,
381412,I 35E,6,"lane_change,lane_change,lane_change,lane_chang..."
382547,I 35E,6,"lane_change,lane_change,lane_change,lane_chang..."
385471,I 35E,6,"lane_change,lane_change,lane_change,lane_chang..."
390634,I 35E,6,"lane_change,lane_change,lane_change,lane_chang..."
390975,I 35E,6,"lane_change,lane_change,lane_change,lane_chang..."


## Transit Feature Change


Transit changes follow the same pattern as highway network changes with a selection and then a change for each project.

In [87]:
trip_ids = ["14941148-JUN19-MVS-BUS-Weekday-01"]
new_headway = 1600
transit_project = {
    "project": "Bus Frequency Blue1",
    "transit_property_change": {
        "service": {
            "trip_properties": {"trip_id": trip_ids}, 
        },
        "property_changes": {"headway_secs": {"set": new_headway}},
    }
}

In [88]:
transit_net_build = transit_net.apply(transit_project)
transit_net_build.feed.frequencies.loc[transit_net_build.feed.frequencies.trip_id.isin(trip_ids)]

,trip_id,headway_secs,start_time,end_time,projects
1,14941148-JUN19-MVS-BUS-Weekday-01,1600,2024-10-16 06:00:00,2024-10-16 09:00:00,"Bus Frequency Blue1,"


# 4 - Write out Networks

Networks can be written by calling their respective `write()` method.  If you'd like to output them to other serialization formats, you can do that by using the `file_format` keyword.

Transit file formats: `csv`, `parquet`, `txt`
Roadway file formats: `geojson`, `shp`, `parquet` 

Note that some fields - noteably those with arrays within the field - are not compatible with some serialization formats.

In [90]:
OUTPATH = Path.cwd()
OUTPATH

PosixPath('/Users/elizabethsall/Documents/GitHub/network_wrangler/notebook')

In [91]:
write_roadway(road_net, prefix="my_net", out_dir=OUTPATH)

In [92]:
write_transit(transit_net, out_dir=OUTPATH)

# 5 - Project cards to document selections, changes, and metadata

The whole property change process above can be more seamless and replicable by using "project cards" which specify  the selection, changes, and project metadata.   

In [94]:
roadway_project_card_file = STPAUL_DIR / "project_cards"/ "road.managed_lane.simple.yml"
roadway_project_card = read_card(roadway_project_card_file)
build_road_net = road_net.apply(roadway_project_card)

# 6 - Scenarios to manage groups of changes

The Scenario object manages groups of projects defined in project cards. 

- Scenarios are initialized by defining a base scenario, which at a minimum specifies the base roadway networks and also may specify a base transit networks and a list of projects that have already been applied.
- Projects can  be "added" or "queued" to a scenario.
- Projects are then "applied" or "built" on the scenario using logic about pre-requisites and co-requisites. 

In [98]:
from network_wrangler import create_scenario
projects = [
    STPAUL_DIR/"project_cards"/"road.add_and_delete.yml",
    STPAUL_DIR/"project_cards"/"road.prop_change.time_of_day.yml",
]
my_scenario = create_scenario(
    base_scenario={"road_net": road_net, "transit_net": transit_net},
    project_card_filepath=projects,
)

my_scenario.apply_all_projects()

# note - see how there is a warning because we have deleted a link that had transit running on it.  This should prompt you to either add a transit re-routing change to that project card or apply another project that will remedy this situation.

Base_scenario doesn't contain ['road_net', 'transit_net', 'applied_projects', 'conflicts']
Creating a deep copy of db object.            This will NOT update any references (e.g. from TransitNetwork)
No links found matching criteria.
Timespan is not in increasing order: ['19:00', '6:00'].            End time will be treated as next day.
Timespan is not in increasing order: ['19:00', '6:00'].            End time will be treated as next day.
Timespan is not in increasing order: ['19:00', '6:00'].            End time will be treated as next day.
Timespan is not in increasing order: ['19:00', '6:00'].            End time will be treated as next day.
Timespan is not in increasing order: ['19:00', '6:00'].            End time will be treated as next day.
Timespan is not in increasing order: ['19:00', '6:00'].            End time will be treated as next day.
Timespan is not in increasing order: ['19:00', '6:00'].            End time will be treated as next day.
Timespan is not in increasing o

In [100]:
my_scenario.summary

['time of day based properties', 'test multiple add and delete']